# Final Project Submission
## Please fill out:

* Student name: Nastaran Nazemian
* Student pace: part time
* Scheduled project review date/time: 28/4/2020 1:30PM (PS)
* Instructor name: Eli Thomas
* Blog post URL:?????????????


## Introduction

In this project, I evaluated the effects of movies’ Genre, Popularity, and IMDb Rate on the domestic and global revenue. For this, I scraped data from IMDb’s website. I‘ve selected 1,500 movies that were released from January 1st 2010 through January 1st 2020 in the USA, which have high Popularity. 

For the first step, I used following libraries to scrape data:
* Beautiful Soup for pulling data
* Requests for sending http requests and download page content
* Datetime for manipulating dates and time and converting strings to date and time
* Json for writing, reading, and saving data in json format
* Time for functions related to working with times
There are 680 pages of movies that were released from January 1st 2010 through January 1st 2020 in the USA. Each page contains up to 50 movies. In total, there are 33,591 movies.

There are 680 pages of movies that were released from January 1st 2010 through January 1st 2020 in the USA. Each page contains up to 50 movies. In total, there are 33,591 movies.


In [1]:
from bs4 import BeautifulSoup
import requests 
from datetime import datetime
from datetime import date
import json
import time

I wrote the `search_movies_page(index)` function to take a page number, scrape the movies from the page, and return a list of movie URLs.

In [2]:
#Read All Movies in One Page
def search_movies_page(index):
    html_page = requests.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2020-01-01&countries=us&start=' + str(index * 50 + 1))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    movies = soup.find_all('div', "lister-item-content")
    urls = []
    for movie in movies:
        movie_url = 'https://www.imdb.com' + movie.find('h3', 'lister-item-header').find('a')['href'] 
        urls.append(movie_url)
    return(urls)

I wrote another function called `movie_page(url)` which takes a movie URL and scrapes all the information related to that movie.

In [3]:
#Read All the Information of One Movie
def movie_page(url):
    movie_page = requests.get(url)    
    return BeautifulSoup(movie_page.content, 'html.parser')


`movie_page(url)` take a movie URL and script all the information of that movie 


In [4]:
#Return a Movie_Id
def movie_id(url):
    title = {}
    movie_title_id = url.split('/')
    title['Title_id'] = movie_title_id[-2]
    return title
#movie_id(url)

When scraping movie details, there might be missing elements. I am using try/except to make sure my program doesn’t break and continues collecting other information. The function `boxoffice(this_movie_page)` is an example with try/except logic. This is the function that collects all the information for a movie. The function calls multiple functions to get different parts of movie’s information, and puts them all together.

In [5]:
#Return the Budget, Gross USA, Cumulative Worldwide Gross, Opening Weekend USA 
def boxoffice(this_movie_page):
    box_office = ['Budget','Gross USA','Cumulative Worldwide Gross','Opening Weekend USA']
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    for x in box_office:
        try:
            finance_value = this_movie_page.find('h4', string= x + ':')
            results[x] = finance_value.parent.contents[2].replace('$',"").replace(',',"").strip()
        except:
            results[x] = None
    return results
# boxoffice(this_movie_page)

In [6]:
#Return Title, Year, Popularity
def movie_info(this_movie_page):
    results = {}
    try:
        results['Title'] = this_movie_page.find('div', class_="title_wrapper").find('h1', class_="").contents[0].replace('\xa0','')
    except:
        results['Title'] = None
    try:
        results['Year'] = this_movie_page.find('div', class_="title_wrapper").find('a').contents[0]  
    except:
        results['Year'] = None
    try:
        results['Popularity'] = this_movie_page.find_all('div', class_ = "titleReviewBarSubItem")[-1].find('span', "subText").contents[0].strip().split(' ')[0].strip().replace(',','')
    except:
        results['Popularity'] = None
    return results

In [7]:
#Return Genres
def movie_genres(this_movie_page):
    genres_out = {}
    genres_list = this_movie_page.find('h4', string = 'Genres:').parent.text.replace('|','').split()
    genres_out['Genre'] = genres_list[1:]
    return genres_out

In [8]:
#Return Runtime
def movie_runtime(this_movie_page):
    runtime_out = {}
    runtime = this_movie_page.find('h4',string="Runtime:").find_next_sibling('time').text.split()
    runtime_out['Runtime_min'] = runtime[0]
    return runtime_out

In [9]:
#Return Release_Day
def movie_release_day(this_movie_page):
    release_day = {}
    release_day_list = this_movie_page.find('div', class_ = "subtext").find('a', title = "See more release dates").text.split()
    date = " ".join(release_day_list[0:3])
    release_day['Release_day'] = datetime.strptime(date, '%d %B %Y').strftime("%m/%d/%Y")
    return release_day

In [10]:
#Return Rating
def rating (this_movie_page):
    results = {}
    try:
        check_rating = this_movie_page.find('div', class_="ratingValue")
        rating = check_rating.find('strong')['title'].split()
        results['IMDB_Rate'] = rating[0]
        results['Rating_count'] = rating[3].replace(',','')
    except:
        results['IMDB_Rate'] = None
        results['Rating_count'] = None
        
    try:  
        check_value = this_movie_page.find('div',"metacriticScore score_mixed titleReviewBarSubItem")
        results['Meta_Score'] = check_value.find('span').text
    except:
        results['Meta_Score'] = None
    return results


In [11]:
#Rturn Casts
def casts_names(this_movie_page):
    roles = [('Director',2),('Writer',2), ('Star',3)]
    results = {}
    for role in roles:
        cast_role = (role[0]+':')

        name_list = []
        try:
            check_role = this_movie_page.find('h4',string=role[0]+':')
            if (check_role is None):
                check_role = this_movie_page.find('h4',string=role[0]+'s:')
            names = check_role.find_next_siblings('a')
            for name in names[:role[1]]:
                name_list.append(name.text)
            results[role[0]] = name_list
        except:
            results[role[0]] = None
    return results


In [12]:
#Find Number_User_Review, Number_Critical_Review
def review (this_movie_page):
    results = {}
    check_value = this_movie_page.find('div', "titleReviewBarItem titleReviewbarItemBorder")
    if check_value is not None:
        review = check_value.find('span', "subText")
        user = review.find('a').text.split()
        results['User_Review_Number'] = user[0].replace(',','')
        critic = review.find('a', href = "externalreviews").text.split()
        results['Critical_Review_Number'] = critic[0]
    else:
        results['User_Review_Number'] = None
        results['Critical_Review_Number'] = None
    return(results)


In [13]:
#Find Meta_rating
def meta_rate (this_movie_page):
    results = {}
    check_value = this_movie_page.find('div', class_="title_wrapper")
    if check_value is not None:
        sub_text = check_value.find('div', class_ = "subtext").text.split('|')
        meta_rating = sub_text[0].strip()
        results['Meta_Rating'] = meta_rating
    else:
        results['Meta_Rating'] = None
    
    return(results)

In [14]:

def one_movie_info(url):
    this_movie_page = movie_page(url)
    movie_data = {}
    try:
        movie_data.update(movie_id(url))
    except:
        print ("ERROR:")
    try:
        movie_data.update(boxoffice(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_info(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_genres(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_runtime(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_release_day(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(rating (this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(casts_names(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(review (this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(meta_rate (this_movie_page))
    except:
        print ("ERROR:")
    
    return(movie_data)  


In [15]:
#Read 100 Page and Save the Result as a JSON File
movies_data = []
for index in range (0, 100):
    urls = search_movies_page(index)
    for url in urls:
        print(url)
#         time.sleep(1)
        one_row = one_movie_info(url)
        movies_data.append(one_row)
print('end', len(movies_data))
with open('data_0427_1222AM_100pages_2010_2020.json', 'w') as outfile:
    json.dump(movies_data, outfile)

https://www.imdb.com/title/tt8367814/
https://www.imdb.com/title/tt7131622/
https://www.imdb.com/title/tt8946378/
https://www.imdb.com/title/tt8579674/
https://www.imdb.com/title/tt1598778/
https://www.imdb.com/title/tt3281548/
https://www.imdb.com/title/tt2527338/
https://www.imdb.com/title/tt6673612/
https://www.imdb.com/title/tt4154796/
https://www.imdb.com/title/tt7286456/
https://www.imdb.com/title/tt7975244/
https://www.imdb.com/title/tt6189022/
https://www.imdb.com/title/tt2584384/
https://www.imdb.com/title/tt8772262/
https://www.imdb.com/title/tt8079248/
https://www.imdb.com/title/tt5727208/
https://www.imdb.com/title/tt7349950/
https://www.imdb.com/title/tt1950186/
https://www.imdb.com/title/tt2066051/
https://www.imdb.com/title/tt1302006/
https://www.imdb.com/title/tt6320628/
https://www.imdb.com/title/tt0816692/
https://www.imdb.com/title/tt1856101/
https://www.imdb.com/title/tt6105098/
https://www.imdb.com/title/tt1375666/
https://www.imdb.com/title/tt4209788/
https://www.

https://www.imdb.com/title/tt5580036/
https://www.imdb.com/title/tt6324278/
https://www.imdb.com/title/tt8267604/
https://www.imdb.com/title/tt0385887/
https://www.imdb.com/title/tt6472976/
https://www.imdb.com/title/tt1343092/
https://www.imdb.com/title/tt2404435/
https://www.imdb.com/title/tt8364368/
https://www.imdb.com/title/tt6398184/
https://www.imdb.com/title/tt4975722/
https://www.imdb.com/title/tt2096673/
https://www.imdb.com/title/tt0780504/
https://www.imdb.com/title/tt3104988/
https://www.imdb.com/title/tt0398286/
https://www.imdb.com/title/tt5083738/
https://www.imdb.com/title/tt1276104/
https://www.imdb.com/title/tt2381941/
https://www.imdb.com/title/tt1840309/
https://www.imdb.com/title/tt4846232/
https://www.imdb.com/title/tt5439796/
https://www.imdb.com/title/tt2239822/
https://www.imdb.com/title/tt2316204/
https://www.imdb.com/title/tt2481498/
https://www.imdb.com/title/tt1673434/
https://www.imdb.com/title/tt0369610/
https://www.imdb.com/title/tt5028340/
https://www.

https://www.imdb.com/title/tt1979320/
https://www.imdb.com/title/tt5719748/
https://www.imdb.com/title/tt8526872/
https://www.imdb.com/title/tt2245084/
https://www.imdb.com/title/tt1458175/
https://www.imdb.com/title/tt6343314/
https://www.imdb.com/title/tt2582846/
https://www.imdb.com/title/tt8385474/
https://www.imdb.com/title/tt1790864/
https://www.imdb.com/title/tt2397535/
https://www.imdb.com/title/tt1413492/
https://www.imdb.com/title/tt4034228/
https://www.imdb.com/title/tt1935859/
https://www.imdb.com/title/tt1213641/
https://www.imdb.com/title/tt7547410/
https://www.imdb.com/title/tt0435761/
https://www.imdb.com/title/tt2872732/
https://www.imdb.com/title/tt5362988/
https://www.imdb.com/title/tt1408101/
https://www.imdb.com/title/tt4160708/
https://www.imdb.com/title/tt1365519/
https://www.imdb.com/title/tt0815236/
ERROR:
https://www.imdb.com/title/tt1615160/
https://www.imdb.com/title/tt4532826/
https://www.imdb.com/title/tt1454468/
https://www.imdb.com/title/tt3332064/
https

https://www.imdb.com/title/tt1343727/
https://www.imdb.com/title/tt9134216/
https://www.imdb.com/title/tt4572514/
https://www.imdb.com/title/tt3717490/
https://www.imdb.com/title/tt1428538/
https://www.imdb.com/title/tt1133985/
https://www.imdb.com/title/tt2788710/
https://www.imdb.com/title/tt1855199/
https://www.imdb.com/title/tt2180411/
https://www.imdb.com/title/tt1411697/
https://www.imdb.com/title/tt8328716/
https://www.imdb.com/title/tt2053463/
https://www.imdb.com/title/tt1706593/
https://www.imdb.com/title/tt1499658/
https://www.imdb.com/title/tt3065204/
https://www.imdb.com/title/tt1854564/
https://www.imdb.com/title/tt6772950/
https://www.imdb.com/title/tt2377322/
https://www.imdb.com/title/tt3165612/
https://www.imdb.com/title/tt1663662/
https://www.imdb.com/title/tt1430132/
https://www.imdb.com/title/tt0964517/
https://www.imdb.com/title/tt3201640/
https://www.imdb.com/title/tt5294518/
https://www.imdb.com/title/tt4257926/
https://www.imdb.com/title/tt0983193/
https://www.

https://www.imdb.com/title/tt5140878/
https://www.imdb.com/title/tt0955308/
https://www.imdb.com/title/tt1220634/
https://www.imdb.com/title/tt9425078/
https://www.imdb.com/title/tt2103281/
https://www.imdb.com/title/tt4786282/
https://www.imdb.com/title/tt5913798/
https://www.imdb.com/title/tt1293847/
https://www.imdb.com/title/tt1321511/
https://www.imdb.com/title/tt1620935/
https://www.imdb.com/title/tt7043012/
https://www.imdb.com/title/tt1001526/
https://www.imdb.com/title/tt3774114/
https://www.imdb.com/title/tt1583421/
https://www.imdb.com/title/tt6957966/
https://www.imdb.com/title/tt1666801/
https://www.imdb.com/title/tt1764651/
https://www.imdb.com/title/tt1155076/
https://www.imdb.com/title/tt1424381/
https://www.imdb.com/title/tt1490017/
https://www.imdb.com/title/tt2304933/
https://www.imdb.com/title/tt3774802/
https://www.imdb.com/title/tt1284575/
https://www.imdb.com/title/tt0427152/
https://www.imdb.com/title/tt2639336/
https://www.imdb.com/title/tt9000224/
https://www.

...

https://www.imdb.com/title/tt6222286/
https://www.imdb.com/title/tt6566768/
https://www.imdb.com/title/tt2980706/
https://www.imdb.com/title/tt3165630/
https://www.imdb.com/title/tt3726704/
https://www.imdb.com/title/tt2179936/
https://www.imdb.com/title/tt3877296/
https://www.imdb.com/title/tt5990444/
https://www.imdb.com/title/tt4382824/
https://www.imdb.com/title/tt2322517/
https://www.imdb.com/title/tt4778988/
https://www.imdb.com/title/tt3704416/
https://www.imdb.com/title/tt4083572/
https://www.imdb.com/title/tt6023242/
https://www.imdb.com/title/tt7052494/
https://www.imdb.com/title/tt2965412/
https://www.imdb.com/title/tt2474976/
https://www.imdb.com/title/tt3454828/
https://www.imdb.com/title/tt7167630/
https://www.imdb.com/title/tt10085338/
https://www.imdb.com/title/tt3896102/
https://www.imdb.com/title/tt5539052/
https://www.imdb.com/title/tt4653586/
https://www.imdb.com/title/tt2167202/
https://www.imdb.com/title/tt2334733/
https://www.imdb.com/title/tt9032888/
https://www

https://www.imdb.com/title/tt5843780/
https://www.imdb.com/title/tt1436568/
https://www.imdb.com/title/tt1582271/
https://www.imdb.com/title/tt10049848/
https://www.imdb.com/title/tt1684555/
https://www.imdb.com/title/tt2458106/
https://www.imdb.com/title/tt5851014/
https://www.imdb.com/title/tt4310022/
https://www.imdb.com/title/tt5464234/
https://www.imdb.com/title/tt2881698/
https://www.imdb.com/title/tt3824412/
https://www.imdb.com/title/tt1716747/
https://www.imdb.com/title/tt3138344/
https://www.imdb.com/title/tt3720788/
https://www.imdb.com/title/tt3520418/
https://www.imdb.com/title/tt8737124/
ERROR:
https://www.imdb.com/title/tt3977462/
https://www.imdb.com/title/tt7089628/
https://www.imdb.com/title/tt5715410/
https://www.imdb.com/title/tt0825283/
https://www.imdb.com/title/tt1742336/
https://www.imdb.com/title/tt5815078/
https://www.imdb.com/title/tt1433822/
https://www.imdb.com/title/tt5314450/
https://www.imdb.com/title/tt7350614/
https://www.imdb.com/title/tt5710042/
http

ERROR:
https://www.imdb.com/title/tt9275046/
https://www.imdb.com/title/tt1090749/
ERROR:
https://www.imdb.com/title/tt5117372/
https://www.imdb.com/title/tt3702652/
https://www.imdb.com/title/tt1838520/
https://www.imdb.com/title/tt4974778/
https://www.imdb.com/title/tt2172985/
https://www.imdb.com/title/tt3663564/
https://www.imdb.com/title/tt7608418/
https://www.imdb.com/title/tt7913934/
ERROR:
https://www.imdb.com/title/tt1988621/
https://www.imdb.com/title/tt3409848/
https://www.imdb.com/title/tt4191702/
https://www.imdb.com/title/tt3093522/
https://www.imdb.com/title/tt3147336/
https://www.imdb.com/title/tt2523832/
ERROR:
https://www.imdb.com/title/tt6613878/
https://www.imdb.com/title/tt2978462/
https://www.imdb.com/title/tt1389098/
https://www.imdb.com/title/tt4351548/
https://www.imdb.com/title/tt3605266/
https://www.imdb.com/title/tt1264115/
https://www.imdb.com/title/tt1978428/
https://www.imdb.com/title/tt3110634/
https://www.imdb.com/title/tt1531924/
https://www.imdb.com/t

https://www.imdb.com/title/tt0851471/
https://www.imdb.com/title/tt2933474/
https://www.imdb.com/title/tt1934452/
https://www.imdb.com/title/tt9485592/
https://www.imdb.com/title/tt9228950/
https://www.imdb.com/title/tt1711018/
https://www.imdb.com/title/tt6495388/
https://www.imdb.com/title/tt11334312/
ERROR:
https://www.imdb.com/title/tt4191580/
https://www.imdb.com/title/tt7066546/
https://www.imdb.com/title/tt6598626/
https://www.imdb.com/title/tt5564792/
https://www.imdb.com/title/tt6923740/
https://www.imdb.com/title/tt1976989/
ERROR:
https://www.imdb.com/title/tt2107861/
https://www.imdb.com/title/tt6595896/
https://www.imdb.com/title/tt6705860/
https://www.imdb.com/title/tt5143890/
https://www.imdb.com/title/tt6105774/
https://www.imdb.com/title/tt3627704/
https://www.imdb.com/title/tt3239442/
https://www.imdb.com/title/tt6649066/
ERROR:
https://www.imdb.com/title/tt9104384/
ERROR:
https://www.imdb.com/title/tt1092634/
https://www.imdb.com/title/tt1965065/
https://www.imdb.com/

ConnectionError: ('Connection aborted.', OSError("(10051, 'WSAENETUNREACH')",))

In [16]:
import re
import numpy as np
import pandas as pd
import time
# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

In [17]:
df = pd.DataFrame(movies_data)
df.head()


,Title_id,Budget,Gross USA,Cumulative Worldwide Gross,Opening Weekend USA,Title,Year,Popularity,Genre,Runtime_min,Release_day,IMDB_Rate,Rating_count,Meta_Score,Director,Writer,Star,User_Review_Number,Critical_Review_Number,Meta_Rating
0,tt8367814,22000000,36296853,114996853,10651884,The Gentlemen,2019,3,"[Action, Comedy, Crime]",113,01/24/2020,7.9,115719,51,[Guy Ritchie],"[Guy Ritchie, Ivan Atkinson]","[Matthew McConaughey, Charlie Hunnam, Michelle...",885,232,R
1,tt7131622,90000000,142502728,374343626,41082018,Once Upon a Time... in Hollywood,2019,5,"[Comedy, Drama]",161,07/26/2019,7.7,448622,None,[Quentin Tarantino],[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie]",4829,594,R
2,tt8946378,40000000,165363234,309232797,26769548,Knives Out,2019,11,"[Comedy, Crime, Drama, Mystery, Thriller]",131,11/27/2019,7.9,283282,None,[Rian Johnson],[Rian Johnson],"[Daniel Craig, Chris Evans, Ana de Armas]",1840,417,PG-13
3,tt8579674,95000000,159227644,368027644,576216,1917,2019,17,"[Drama, War]",119,01/10/2020,8.3,289839,None,[Sam Mendes],"[Sam Mendes, Krysty Wilson-Cairns]","[Dean-Charles Chapman, George MacKay, Daniel M...",2481,449,R
4,tt1598778,60000000,75658097,136515867,22403596,Contagion,2011,24,"[Action, Drama, Thriller]",106,09/09/2011,6.7,258645,None,[Steven Soderbergh],[Scott Z. Burns],"[Matt Damon, Kate Winslet, Jude Law]",685,256,PG-13
